# AAAIM Evaluation Test - NCBI Gene

This notebook tests both single model evaluation and batch evaluation of multiple models.

In [6]:
import os
import sys
import pandas as pd
import numpy as np
from pathlib import Path
import logging

# Add the project root to the Python path
project_root = Path().absolute().parent
sys.path.insert(0, str(project_root))

# Import AAAIM functions
from core import annotate_model, curate_model
from utils.evaluation import (
    evaluate_single_model,
    evaluate_models_in_folder,
    print_evaluation_results,
    compare_results
)

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# LLM configuration
# llm_model = "meta-llama/llama-3.3-70b-instruct:free"  # or "gpt-4o-mini"
llm_model = "meta-llama/llama-3.3-70b-instruct"

# Evaluation parameters
max_entities_per_model = 10  # Limit entities per model for testing
num_models_to_test = 5  # Number of models to test in batch evaluation

# Entity and database configuration
entity_type = "gene"
database = "ncbigene"

output_dir = "./results/"  # Output directory for results

## Annotating a single SBML-FBC model

In [2]:
test_model_file = "/Users/luna/Desktop/CRBM/AMAS_proj/Models/BioModels/BIOMD0000001046.xml"
# Check if test model exists
if os.path.exists(test_model_file):
    print(f"✓ Test model found: {test_model_file}")
else:
    print(f"✗ Test model not found: {test_model_file}")

✓ Test model found: /Users/luna/Desktop/CRBM/AMAS_proj/Models/BioModels/BIOMD0000001046.xml


In [6]:
# Test with a single model
recommendations_df, metrics = annotate_model(
    model_file=test_model_file,
    llm_model=llm_model,
    max_entities=max_entities_per_model,
    entity_type=entity_type,
    database=database,
    tax_id=83332
)

2025-06-09 17:08:01,924 - INFO - Starting annotation for model: /Users/luna/Desktop/CRBM/AMAS_proj/Models/BioModels/BIOMD0000001046.xml
2025-06-09 17:08:01,925 - INFO - Using LLM model: meta-llama/llama-3.3-70b-instruct:free
2025-06-09 17:08:01,925 - INFO - Using method: direct for database search
2025-06-09 17:08:01,926 - INFO - Entity type: gene, Database: ncbigene
2025-06-09 17:08:01,926 - INFO - Using organism-specific search for tax_id: 83332
2025-06-09 17:08:01,926 - INFO - Step 1: Getting all species from model...
2025-06-09 17:08:02,004 - INFO - Found 28 species in model
2025-06-09 17:08:02,069 - INFO - Found 25 entities with existing annotations
2025-06-09 17:08:02,069 - INFO - Selected 10 entities for annotation
2025-06-09 17:08:02,069 - INFO - Step 4: Extracting model context...
2025-06-09 17:08:02,190 - INFO - Extracted context for model: M_MAP
2025-06-09 17:08:02,190 - INFO - Step 5: Formatting LLM prompt...
2025-06-09 17:08:02,310 - INFO - Step 6: Querying LLM (meta-llama

In [7]:
recommendations_df

,file,type,id,display_name,annotation,annotation_label,match_score,existing,update_annotation
0,BIOMD0000001046.xml,gene,G_accA3,G_accA3,,,0.000000,0,ignore
1,BIOMD0000001046.xml,gene,G_accD3,G_accD3,,,0.000000,0,ignore
2,BIOMD0000001046.xml,gene,G_accD4,G_accD4,,,0.000000,0,ignore
3,BIOMD0000001046.xml,gene,G_accD5,G_accD5,,,0.000000,0,ignore
4,BIOMD0000001046.xml,gene,G_acpS,G_acpS,888626,acpS,0.333333,0,ignore
5,BIOMD0000001046.xml,gene,G_acpS,G_acpS,887052,acpA,0.333333,0,ignore
6,BIOMD0000001046.xml,gene,G_birA,G_birA,888726,birA,0.333333,0,ignore
7,BIOMD0000001046.xml,gene,G_cmaA2,G_cmaA2,887264,cmaA2,0.333333,0,ignore
8,BIOMD0000001046.xml,gene,G_desA1,G_desA1,885444,desA1,0.666667,0,add
9,BIOMD0000001046.xml,gene,G_desA2,G_desA2,,,0.000000,0,ignore


In [8]:
metrics

{'total_entities': 28,
 'entities_with_predictions': 4,
 'annotation_rate': 0.14285714285714285,
 'total_predictions': 5,
 'matches': 0,
 'accuracy': 0.0,
 'total_time': 5.8827431201934814,
 'llm_time': 5.420718193054199,
 'search_time': 0.07085084915161133}

## Test 1: Single Model Evaluation

Evaluation of a single model with existing annotations.

In [7]:
# Test using utils evaluation function
result_df = evaluate_single_model(
    model_file=test_model_file,
    tax_id=83332,
    llm_model=llm_model,
    # max_entities=max_entities_per_model,
    entity_type=entity_type,
    database=database,
    save_llm_results=True,
    output_dir=output_dir
)

2025-06-09 17:22:58,826 - INFO - Evaluating model: BIOMD0000001046.xml
2025-06-09 17:22:58,827 - INFO - Using organism-specific search for tax_id: 83332
2025-06-09 17:22:58,905 - INFO - Evaluating 25 entities in BIOMD0000001046.xml
2025-06-09 17:23:00,580 - INFO - HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"


LLM results saved to: results/llama-3.3-70b-instruct/gene/BIOMD0000001046.txt


In [ ]:
result_df

,model,species_id,display_name,synonyms_LLM,reason,exist_annotation_id,exist_annotation_name,predictions,predictions_names,match_score,recall_formula,precision_formula,recall_exact,precision_exact,accuracy,total_time,llm_time,query_time
0,BIOMD0000001046.xml,G_accA3,accA,"[accA, ACACA, ACACB]",The provided genes appear to be involved in va...,[887912],accA3,[],[],[],0,0,0.0,0.0,0,13.547572,13.536939,0.010633
1,BIOMD0000001046.xml,G_accD3,accD,"[accD, ACCD, ACC]",The provided genes appear to be involved in va...,[885279],accD3,[],[],[],0,0,0.0,0.0,0,13.547572,13.536939,0.010633
2,BIOMD0000001046.xml,G_accD4,accD,"[accD, ACCD, ACC]",The provided genes appear to be involved in va...,[886131],accD4,[],[],[],0,0,0.0,0.0,0,13.547572,13.536939,0.010633
3,BIOMD0000001046.xml,G_accD5,accD,"[accD, ACCD, ACC]",The provided genes appear to be involved in va...,[888725],accD5,[],[],[],0,0,0.0,0.0,0,13.547572,13.536939,0.010633
4,BIOMD0000001046.xml,G_acpS,acpS,"[acpS, ACP, ACPS]",The provided genes appear to be involved in va...,[888626],acpS,[888626],[acpS],[0.6666666666666666],0,0,1.0,1.0,1,13.547572,13.536939,0.010633
5,BIOMD0000001046.xml,G_cmaA2,cmaA2,"[cmaA2, CMA1, CMA2]",The provided genes appear to be involved in va...,[887264],cmaA2,[887264],[cmaA2],[0.6666666666666666],0,0,1.0,1.0,1,13.547572,13.536939,0.010633
6,BIOMD0000001046.xml,G_desA1,desA,"[desA, DES, DESA]",The provided genes appear to be involved in va...,[885444],desA1,[885444],[desA1],[0.3333333333333333],0,0,1.0,1.0,1,13.547572,13.536939,0.010633
7,BIOMD0000001046.xml,G_desA2,desA,"[desA, DES, DESA]",The provided genes appear to be involved in va...,[885339],desA2,[885444],[desA1],[0.3333333333333333],0,0,0.0,0.0,0,13.547572,13.536939,0.010633
8,BIOMD0000001046.xml,G_desA3,desA,"[desA, DES, DESA]",The provided genes appear to be involved in va...,[888821],desA3,[885444],[desA1],[0.3333333333333333],0,0,0.0,0.0,0,13.547572,13.536939,0.010633
9,BIOMD0000001046.xml,G_fabD,fabD,"[fabD, FABD, FADB]",The provided genes appear to be involved in va...,[888769],fabD,"[888769, 885799]","[fabD, fadB]","[0.6666666666666666, 0.3333333333333333]",0,0,1.0,0.5,1,13.547572,13.536939,0.010633


In [9]:
result_df.to_csv(output_dir+"biomd_ncbigene_direct_llama_1046.csv")

In [ ]:
test_model_file = "/Users/luna/Desktop/CRBM/AMAS_proj/Models/BioModels/BIOMD0000001062.xml"
result_df = evaluate_single_model(
    model_file=test_model_file,
    tax_id=83332,
    llm_model=llm_model,
    # max_entities=max_entities_per_model,
    entity_type=entity_type,
    database=database,
    save_llm_results=True,
    output_dir=output_dir
)

## Test 2: Batch Model Evaluation

Test the evaluation of multiple models in a directory.

In [13]:
# model_dir = "/Users/luna/Desktop/CRBM/AMAS_proj/Models/BioModels"
model_dir = "test_models"
# Check if model directory exists
if os.path.exists(model_dir):
    model_files = [f for f in os.listdir(model_dir) if f.endswith('.xml')]
    print(f"✓ Model directory found: {model_dir}")
    print(f"  - Found {len(model_files)} XML files")
    print(f"  - Will test first {min(num_models_to_test, len(model_files))} models")

✓ Model directory found: test_models
  - Found 3 XML files
  - Will test first 3 models


In [6]:
# Run batch evaluation 
batch_results_df = evaluate_models_in_folder(
    model_dir=model_dir,
    # num_models=min(num_models_to_test, len(model_files)),
    llm_model=llm_model,
    # max_entities=max_entities_per_model,
    entity_type=entity_type,
    database=database,
    save_llm_results=True,
    output_dir=output_dir,
    output_file="batch_evaluation_results.csv",
    start_at=1
)
        
if not batch_results_df.empty:
    print(f"✓ Batch evaluation successful")
    print(f"  - Evaluated {batch_results_df['model'].nunique()} models")
    print(f"  - Generated {len(batch_results_df)} total result rows")
    print(f"  - Average accuracy: {batch_results_df['accuracy'].mean():.1%}")
    
    # Show updated metrics
    print("\n  Updated metrics summary:")
    if 'recall_formula' in batch_results_df.columns:
        print(f"  - Average recall (formula): {batch_results_df['recall_formula'].mean():.3f}")
        print(f"  - Average precision (formula): {batch_results_df['precision_formula'].mean():.3f}")
    print(f"  - Average recall (ChEBI): {batch_results_df['recall_chebi'].mean():.3f}")
    print(f"  - Average precision (ChEBI): {batch_results_df['precision_chebi'].mean():.3f}")
    
    # Show sample of LLM results
    print("\n  Sample LLM results:")
    if 'synonyms_LLM' in batch_results_df.columns:
        for idx, row in batch_results_df[['species_id', 'synonyms_LLM']].head(3).iterrows():
            print(f"    {row['species_id']}: {row['synonyms_LLM']}")
    
    # Show match scores instead of predictions_hits
    print("\n  Sample match scores:")
    if 'match_score' in batch_results_df.columns:
        for idx, row in batch_results_df[['species_id', 'match_score']].head(3).iterrows():
            print(f"    {row['species_id']}: {row['match_score']}")
    
    # Print summary statistics
    print("\n  Summary statistics:")
    print_evaluation_results(os.path.join(output_dir, "batch_evaluation_results.csv"))
    
else:
    print(f"✗ Batch evaluation failed: No results generated")
            

2025-05-25 16:31:32,475 - INFO - Evaluating 3 models starting from index 1
2025-05-25 16:31:32,476 - INFO - Evaluating 1/3: BIOMD0000000190.xml
2025-05-25 16:31:32,476 - INFO - Evaluating model: BIOMD0000000190.xml
2025-05-25 16:31:32,494 - INFO - Evaluating 10 entities in BIOMD0000000190.xml



2.1 Running batch evaluation
--------------------------------------------------


2025-05-25 16:31:33,726 - INFO - HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
2025-05-25 16:31:47,641 - INFO - Saved intermediate results to: results/batch_evaluation_results.csv_1.csv
2025-05-25 16:31:47,642 - INFO - Evaluating 2/3: BIOMD0000000508.xml
2025-05-25 16:31:47,642 - INFO - Evaluating model: BIOMD0000000508.xml
2025-05-25 16:31:47,646 - INFO - Evaluating 5 entities in BIOMD0000000508.xml


LLM results saved to: results/llama-3.3-70b-instruct/BIOMD0000000190_llm_results.txt


2025-05-25 16:31:50,066 - INFO - HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
2025-05-25 16:31:52,557 - INFO - Saved intermediate results to: results/batch_evaluation_results.csv_2.csv
2025-05-25 16:31:52,557 - INFO - Evaluating 3/3: BIOMD0000000634.xml
2025-05-25 16:31:52,557 - INFO - Evaluating model: BIOMD0000000634.xml
2025-05-25 16:31:52,574 - INFO - Evaluating 5 entities in BIOMD0000000634.xml


LLM results saved to: results/llama-3.3-70b-instruct/BIOMD0000000508_llm_results.txt


2025-05-25 16:31:54,555 - INFO - HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
2025-05-25 16:32:03,496 - INFO - Saved intermediate results to: results/batch_evaluation_results.csv_3.csv
2025-05-25 16:32:03,498 - INFO - Saved final results to: results/batch_evaluation_results.csv


LLM results saved to: results/llama-3.3-70b-instruct/BIOMD0000000634_llm_results.txt
✓ Batch evaluation successful
  - Evaluated 3 models
  - Generated 20 total result rows
  - Average accuracy: 80.0%

  Updated metrics summary:
  - Average recall (formula): 0.800
  - Average precision (formula): 0.750
  - Average recall (ChEBI): 0.800
  - Average precision (ChEBI): 0.424

  Sample LLM results:
    SAM: ['S-adenosylmethionine', 'AdoMet', 'SAMe']
    A: ['S-adenosylmethioninamine', 'decarboxylated S-adenosylmethionine', 'dcAdoMet']
    P: ['putrescine', '1,4-diaminobutane', 'butane-1,4-diamine']

  Sample match scores:
    SAM: [1.0, 0.3333333333333333, 0.6666666666666666]
    A: [1.0]
    P: [1.0, 0.3333333333333333]

  Summary statistics:
Number of models assessed: 3
Number of models with predictions: 3
Average accuracy (per model): 0.77
Ave. total time (per model): 10.20
Ave. total time (per element, per model): 1.53
Ave. LLM time (per model): 9.76
Ave. LLM time (per element, per mod

## Functions test - FBC model

In [1]:
import os
import sys
import pandas as pd
import numpy as np
from pathlib import Path
import logging
import libsbml

# Add the project root to the Python path
project_root = Path().absolute().parent
sys.path.insert(0, str(project_root))

# Import AAAIM functions
from core import annotate_model, curate_model, model_info

In [3]:
model_file = "/Users/luna/Desktop/CRBM/AMAS_proj/Models/BioModels/BIOMD0000001046.xml"
entity_type = "gene"
species_ids = model_info.get_all_species_ids(model_file, entity_type)
model_info.extract_model_info(model_file, species_ids, entity_type)

{'model_name': 'Raman2005 - Mycolic acid pathway of M. tuberculosis',
 'display_names': {'G_accA3': 'G_accA3',
  'G_accD3': 'G_accD3',
  'G_accD4': 'G_accD4',
  'G_accD5': 'G_accD5',
  'G_acpS': 'G_acpS',
  'G_birA': 'G_birA',
  'G_cmaA2': 'G_cmaA2',
  'G_desA1': 'G_desA1',
  'G_desA2': 'G_desA2',
  'G_desA3': 'G_desA3',
  'G_fabD': 'G_fabD',
  'G_fabG1_mabA': 'G_fabG1_mabA',
  'G_fabG2': 'G_fabG2',
  'G_fabG4': 'G_fabG4',
  'G_fabH': 'G_fabH',
  'G_fadD32': 'G_fadD32',
  'G_fas': 'G_fas',
  'G_gene42160': 'G_gene42160',
  'G_inhA': 'G_inhA',
  'G_kasB': 'G_kasB',
  'G_kasA': 'G_kasA',
  'G_mmaA1': 'G_mmaA1',
  'G_mmaA2': 'G_mmaA2',
  'G_mmaA3': 'G_mmaA3',
  'G_mmaA4': 'G_mmaA4',
  'G_pcaA': 'G_pcaA',
  'G_pks13': 'G_pks13',
  'G_UNK2': 'G_UNK2'},
 'reactions': [],
 'model_notes': 'The mycobacterial cell wall is a distinctive thick layer that protects the tubercle bacillus from general antibiotics and the host’s immune system. Mycolic acids, which are long-chain α-alkyl-β-hydroxy fatty

In [11]:
model_file = "/Users/luna/Desktop/CRBM/AMAS_proj/Models/BioModels/BIOMD0000001046.xml"
reader = libsbml.SBMLReader()
document = reader.readSBML(model_file)
model = document.getModel()
fbc_plugin = model.getPlugin("fbc")
fbc_plugin.getPackageName()

'fbc'

In [72]:
for i in range(model.getNumReactions()):
    reaction = model.getReaction(i)
    print(reaction.getIdAttribute())
    GeneProduct = reaction.getPlugin("fbc").getGeneProductAssociation()
    if GeneProduct:
        print(GeneProduct.getAssociation().toInfix())
    else:
        print("No gene product in reaction: ", reaction.getIdAttribute())


J001
G_acpS
J002
G_birA
J003
G_accA3
J004
G_accD3
J005
G_fas
J006
G_fas
J007
G_fas
J008
G_fas
J009
G_fas
J010
G_fas
J011
G_fas
J012
G_fas
J013
G_fas
J014
G_fas
J015
G_fas
J016
G_fas
J017
G_fas
J018
G_fas
J019
G_fas
J020
G_fas
J021
G_fas
J022
G_fas
J023
G_fas
J024
G_fas
J025
G_fas
J026
G_fas
J027
G_fas
J028
G_fas
J029
G_fas
J030
G_fas
J031
G_fas
J032
G_fas
J033
G_fas
J034
G_fas
J035
G_fas
J036
G_fas
J037
G_fas
J038
G_fas
J039
G_fas
J040
G_fas
J041
G_fas
J042
G_fas
J043
G_fas
J044
G_fas
J045
G_fas
J046
G_fas
J047
G_fas
J048
G_fas
J049
G_fas
J050
G_fas
J051
G_fas
J052
G_fas
J053
G_fas
J054
G_fas
J055
G_fas
J056
G_fas
J057
G_fas
J058
G_fas
J059
G_fas
J060
G_fas
J061
G_fas
J062
G_fas
J063
G_fabD
J064
G_fabH
J065
G_fabG1_mabA
J066
G_fabG2
J067
G_fabG4
J068
G_gene42160
J069
G_inhA
J070
(G_kasB and G_kasA)
J071
G_fabG1_mabA
J072
G_fabG2
J073
G_fabG4
J074
G_gene42160
J075
G_inhA
J076
(G_kasB and G_kasA)
J077
G_fabG1_mabA
J078
G_fabG2
J079
G_fabG4
J080
G_gene42160
J081
G_inhA
J082
(G_kasB and G_

In [85]:
reaction = model.getReaction("J003")
print(reaction.getIdAttribute())
print("Reactant:")
for reactant in reaction.getListOfReactants():
    species = reactant.getSpecies()
    print(model.getSpecies(species).getName())
print("Product:")
for product in reaction.getListOfProducts():
    species = product.getSpecies()
    print(model.getSpecies(species).getName())

J003
Reactant:
ATP
BCCP-biotin
HCO3-
Product:
ADP
phosphate
BCCP-biotin-CO2
H+


## Functions test - qual model

In [2]:
model_file = "/Users/luna/Desktop/CRBM/AMAS_proj/Models/BioDivine/111_APOPTOSIS_source.sbml"
entity_type = "gene"
species_ids = model_info.get_all_species_ids(model_file, entity_type)
model_info.extract_model_info(model_file, species_ids, entity_type)

Skipping self loop: sa2 = sa2
Skipping self loop: sa27 = sa27
Skipping self loop: sa3 = sa3
Skipping self loop: sa42 = sa42
Skipping self loop: sa43 = sa43
Skipping self loop: sa44 = sa44
Skipping self loop: sa47 = sa47
Skipping self loop: sa48 = sa48
Skipping self loop: sa69 = sa69
Skipping self loop: sa72 = sa72
Skipping self loop: sa73 = sa73
Skipping self loop: sa74 = sa74
Skipping self loop: sa75 = sa75
Skipping self loop: sa76 = sa76
Skipping self loop: sa77 = sa77


{'model_name': 'model_id',
 'model_type': <ModelType.SBML_QUAL: 'SBML-qual'>,
 'format_info': {'has_fbc': False,
  'has_qual': True,
  'num_species': 0,
  'num_reactions': 0,
  'num_qualitative_species': 33,
  'num_transitions': 18},
 'display_names': {'sa27': 'APAF1',
  'csa11': 'BCL2/MCL1/BCL2L1_complex',
  'csa6': 'TRADD/FADD_complex',
  'sa89': 'BID',
  'sa40': 'CASP7',
  'sa48': 'Orf3a',
  'sa17': 'CASP3',
  'csa1': 'TNF/TNFRSF1A_complex',
  'csa5': 'Apoptosome_complex',
  'csa3': 'BAD/BBC3/BCL2L11_complex',
  'sa69': 'E',
  'csa2': 'FAS/FASL_complex',
  'sa44': 'TRADD',
  'sa72': 'Orf3b',
  'sa14': 'CASP8',
  'sa77': 'Orf9b',
  'sa75': 'Orf6',
  'sa42': 'M',
  'sa25': 'CYCS',
  'sa3': 'TNF',
  'sa76': 'S',
  'sa19': 'CASP9_cell_active',
  'sa29': 'AKT1',
  'sa31': 'BAX',
  'sa74': 'N',
  'sa10': 'FADD',
  'sa2': 'FASLG',
  'sa50': 'MAPK14',
  'sa41': 'Apoptosis_phenotype',
  'sa73': 'Orf8a',
  'sa47': 'CASP9_cell',
  'sa43': 'Orf7a',
  'sa79': 'BAD'},
 'reactions': ['csa1 = sa3',

In [4]:
prompt = model_info.format_prompt(model_file, species_ids, entity_type)
print(prompt)

Now annotate these:
Gene to annotate: csa11, csa2, csa1, csa3, csa6, csa5, sa29, sa73, sa27, sa10, sa2, sa69, sa79, sa31, sa3, sa42, sa48, sa50, sa25, sa72, sa47, sa40, sa76, sa19, sa44, sa17, sa14, sa77, sa74, sa41, sa43, sa75, sa89
Model: "model_id" 
// Display Names:
{'sa17': 'CASP3', 'sa72': 'Orf3b', 'sa76': 'S', 'sa73': 'Orf8a', 'sa3': 'TNF', 'sa40': 'CASP7', 'sa19': 'CASP9_cell_active', 'sa31': 'BAX', 'csa3': 'BAD/BBC3/BCL2L11_complex', 'csa2': 'FAS/FASL_complex', 'sa29': 'AKT1', 'sa14': 'CASP8', 'csa5': 'Apoptosome_complex', 'sa47': 'CASP9_cell', 'sa10': 'FADD', 'sa75': 'Orf6', 'sa50': 'MAPK14', 'sa27': 'APAF1', 'csa6': 'TRADD/FADD_complex', 'csa1': 'TNF/TNFRSF1A_complex', 'sa2': 'FASLG', 'sa43': 'Orf7a', 'sa25': 'CYCS', 'sa74': 'N', 'sa41': 'Apoptosis_phenotype', 'sa48': 'Orf3a', 'csa11': 'BCL2/MCL1/BCL2L1_complex', 'sa69': 'E', 'sa89': 'BID', 'sa42': 'M', 'sa79': 'BAD', 'sa77': 'Orf9b', 'sa44': 'TRADD'}
// Boolean Transitions:
sa2 = sa2
sa27 = sa27
sa3 = sa3
sa42 = sa42
sa43 =

In [86]:
reader = libsbml.SBMLReader()
document = reader.readSBML(model_file)
model = document.getModel().getPlugin("qual")
model.getPackageName()

'qual'

In [93]:
transit = model.getTransition("tr_csa1")
transit.getListOfInputs()

<ListOfInputs[6]>

In [92]:
for transition in model.getListOfTransitions():
    print(transition.getId())
    print(transition.getListOfFunctionTerms())
    for function_term in transition.getListOfFunctionTerms():
        print(function_term.getName())

tr_csa1
<ListOfFunctionTerms[1]>

tr_csa2
<ListOfFunctionTerms[1]>

tr_csa4
<ListOfFunctionTerms[1]>

tr_csa3
<ListOfFunctionTerms[1]>

tr_sa4
<ListOfFunctionTerms[1]>

tr_sa5
<ListOfFunctionTerms[1]>

tr_sa12
<ListOfFunctionTerms[1]>

tr_sa16
<ListOfFunctionTerms[1]>

tr_sa17
<ListOfFunctionTerms[1]>

tr_sa18
<ListOfFunctionTerms[1]>

tr_sa33
<ListOfFunctionTerms[1]>

tr_sa34
<ListOfFunctionTerms[1]>

tr_sa36
<ListOfFunctionTerms[1]>

